In [33]:
%pip install numpy tensorflow matplotlib language-tool-python

Note: you may need to restart the kernel to use updated packages.


In [34]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import language_tool_python

In [35]:
# Función para corregir la gramática de una oración
def corregir_gramatica(oracion):
    tool = language_tool_python.LanguageToolPublicAPI('es')
    matches = tool.check(oracion)

    # Aplicar correcciones sugeridas
    oracion_corregida = tool.correct(oracion)

    return oracion_corregida

# Función para corregir la sintaxis de una oración
def corregir_sintaxis(oracion, modelo_sintaxis, tokenizer, max_length):
    # Preprocesar la oración
    oracion_procesada = tokenizer.texts_to_sequences([oracion])
    input_sequence = pad_sequences(oracion_procesada, padding='post', maxlen=max_length)

    # Obtener predicciones del modelo de corrección sintáctica
    predicciones = modelo_sintaxis.predict(input_sequence)

    # Convertir las predicciones a texto
    predicciones_texto = " ".join([tokenizer.index_word[idx] for idx in np.argmax(predicciones[0], axis=-1)])

    return predicciones_texto

# Función para preprocesar datos
def preprocesar_datos(oraciones):
    tokenizer = Tokenizer(oov_token="<OOV>")
    tokenizer.fit_on_texts(oraciones)

    secuencias = tokenizer.texts_to_sequences(oraciones)
    secuencias_padded = pad_sequences(secuencias, padding='post')

    vocab_size = len(tokenizer.word_index) + 1

    return secuencias_padded, vocab_size, tokenizer




# Función para diseñar el modelo
def diseñar_modelo(vocab_size, max_length):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=64, input_length=max_length),
        LSTM(128, return_sequences=True),
        Dense(vocab_size, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

# Función para entrenar el modelo
def entrenar_modelo(model, input_sequences, target_sequences, epochs=10):
    model.fit(input_sequences, target_sequences, epochs=epochs, verbose=2)

# Función para evaluar el modelo
def evaluar_modelo(model, input_sequence, tokenizer):
    predicciones = model.predict(input_sequence)
    
    # Convertir predicciones a texto
    predicciones_texto = " ".join([tokenizer.index_word[idx] for idx in np.argmax(predicciones[0], axis=-1)])

    return predicciones_texto

In [36]:
# Ejemplo de oración a corregir
oracion_original = "Ayer, fui al parque a jugar con mis amigos y me olvidé mis sapatos en casa."


In [37]:
oraciones_correctas = [
    "La puesta de sol sobre el horizonte pintaba el cielo de tonos cálidos y dorados.",
    "Después de una larga jornada de trabajo, disfruto de relajarme con un buen libro.",
    "El aroma fresco de las flores primaverales impregnaba el jardín.",
    "Con cada paso, el sendero serpenteaba a través del bosque, invitándome a explorar.",
    "La melodía suave del piano llenaba la sala, creando una atmósfera tranquila.",
    "Observar las estrellas en una noche despejada siempre me llena de asombro.",
    "El aroma tentador del café recién hecho llenaba la cocina.",
    "Las olas del océano rompían suavemente en la orilla, creando un sonido relajante.",
    "La arquitectura antigua del pueblo añadía un encanto especial a sus calles empedradas.",
    "La risa de los niños resonaba en el parque mientras disfrutaban de su tiempo de juego.",
    "El río serpenteaba entre las montañas, creando un paisaje impresionante.",
    "Después de la lluvia, las gotas brillaban como diamantes en las hojas de los árboles.",
    "El aroma a pan recién horneado llenaba la panadería, tentando a los transeúntes.",
    "La risa contagiosa de mi mejor amigo siempre alegra mi día.",
    "Las luces parpadeantes de la ciudad creaban un espectáculo nocturno fascinante.",
    "El silencio de la biblioteca se rompía solo por el suave murmullo de las páginas que se pasaban.",
    "La brisa marina acariciaba suavemente mi rostro mientras caminaba por la playa.",
    "El jardín estaba lleno de colores vibrantes a medida que las flores estallaban en plena floración.",
    "El sonido de las hojas crujientes bajo mis pies indicaba la llegada del otoño.",
    "Las montañas se erguían majestuosas, cubiertas de un manto de nieve fresca.",
    "El canto de los pájaros anunciaba la llegada de la primavera en el aire.",
    "La chimenea crepitaba, proporcionando calor y un ambiente acogedor en la sala.",
    "El anciano observaba con nostalgia las fotografías que contaban la historia de su vida.",
    "El aroma a canela y manzanas llenaba la cocina durante la preparación de un pastel.",
    "Las estrellas brillaban intensamente en la oscuridad, formando constelaciones en el cielo nocturno.",
    "La arquitectura moderna del edificio contrastaba con los edificios antiguos que lo rodeaban.",
    "El zumbido suave de los grillos creaba una sinfonía nocturna en el campo.",
    "Los colores vivos de un atardecer pintaban el cielo como un lienzo en llamas.",
    "La cascada rugía con fuerza, lanzando gotas de agua en el aire a su alrededor.",
    "El perfume dulce de las rosas embriagaba el jardín en una tarde de verano.",
    "El ruido de las olas rompiendo contra las rocas resonaba en la costa tranquila.",
    "La risa de los niños en el parque resonaba como campanas alegres.",
    "El viento susurraba entre los árboles, creando una sinfonía natural en el bosque.",
    "El anciano relataba historias fascinantes sobre tiempos pasados mientras todos escuchábamos con atención.",
    "La luz tenue de las velas creaba un ambiente romántico en la cena a la luz de la luna.",
    "El café caliente reconfortaba mis manos mientras observaba la lluvia desde la ventana.",
    "Las mariposas danzaban en el aire, agregando un toque de magia al prado.",
    "El rugido lejano de los truenos anunciaba una tormenta inminente en el horizonte.",
    "El aroma a tierra mojada después de la lluvia llenaba el aire con frescura.",
    "Las hojas crujían bajo mis pies mientras paseaba por el bosque en pleno otoño.",
    "El rincón del jardín estaba lleno de vida con la visita de colibríes y mariposas.",
    "La ciudad cobraba vida por la noche, con luces brillantes iluminando los rascacielos.",
    "El chirrido suave de las puertas de madera añadía un toque nostálgico al antiguo caserío.",
    "La neblina matutina envolvía el paisaje en un misterio etéreo.",
    "El murmullo del arroyo acompañaba el tranquilo paseo por el sendero del bosque.",
    "El suave tintineo de campanillas anunciaba la llegada de la brisa primaveral.",
    "La mirada cómplice entre ellos revelaba un amor que resistía el paso del tiempo.",
    "El ulular de los búhos rompía el silencio de la noche en el bosque oscuro.",
    "Las cometas flotaban en el cielo, pintando estelas de colores contra el azul claro.",
    "El sabor salado del aire marino era refrescante mientras caminaba por el muelle.",
    "Las sombras alargadas de los árboles bailaban en el suelo al atardecer.",
    "El sonido de las olas acariciando la playa era una melodía relajante para los oídos.",
    "Las risas resonaban en la plaza, donde la gente disfrutaba de la feria del pueblo.",
    "El calor del sol acariciaba mi piel mientras me recostaba en la hierba del prado.",
    "El murmullo constante de la multitud creaba un ambiente animado en el mercado.",
    "La luna llena iluminaba la noche con su resplandor plateado.",
    "El croar de las ranas acompañaba la noche en el pantano tranquilo.",
    "El aroma a lavanda impregnaba el aire en los campos violetas bajo el sol radiante.",
    "Las hojas danzaban en el viento, creando un remolino de colores en el otoño."
]
oraciones_incorrectas = [
    "Puesta sol horizonte pintar cielo tonos cálidos dorados.",
    "Después una larga jornada trabajo, disfruto relajarme buen libro.",
    "Aroma fresco flores primaverales impregnar jardín.",
    "Con cada paso, sendero serpenteaba través bosque, invitándome explorar.",
    "Melodía suave piano llenar sala, creando atmósfera tranquila.",
    "Observar estrellas en noche despejada siempre llenarme asombro.",
    "Aroma tentador café recién hecho llenar cocina.",
    "Olas océano rompían suavemente orilla, creando sonido relajante.",
    "Arquitectura antigua pueblo añadir encanto especial calles empedradas.",
    "Risa niños resonar parque disfrutar tiempo juego.",
    "Río serpenteaba montañas, creando paisaje impresionante.",
    "Después lluvia, gotas brillar como diamantes hojas árboles.",
    "Aroma pan recién horneado llenar panadería, tentando transeúntes.",
    "Risa contagiosa mi mejor amigo siempre alegrar día.",
    "Luces parpadeantes ciudad crear espectáculo nocturno fascinante.",
    "Silencio biblioteca romperse solo suave murmullo páginas pasar.",
    "Brisa marina acariciar suavemente rostro mientras caminar playa.",
    "Jardín estar lleno colores vibrantes medida flores estallar plena floración.",
    "Sonido hojas crujientes bajo pies indicar llegada otoño.",
    "Montañas erguir majestuosas, cubiertas manto nieve fresca.",
    "Canto pájaros anunciar llegada primavera aire.",
    "Chimenea crepitar, proporcionando calor ambiente acogedor sala.",
    "Anciano observar nostalgia fotografías contar historia vida.",
    "Aroma canela manzanas llenar cocina durante preparación pastel.",
    "Estrellas brillar intensamente en oscuridad, formando constelaciones cielo nocturno.",
    "Arquitectura moderna edificio contrastar edificios antiguos rodear.",
    "Zumbido suave grillos crear sinfonía nocturna campo.",
    "Colores vivos atardecer pintar cielo como lienzo llamas.",
    "Cascada rugir fuerza, lanzando gotas agua aire alrededor.",
    "Perfume dulce rosas embriagar jardín tarde verano.",
    "Ruido olas rompiendo contra rocas resonar costa tranquila.",
    "Risa niños parque resonar como campanas alegres.",
    "Viento susurrar entre árboles, creando sinfonía natural bosque.",
    "Anciano relatar historias fascinantes sobre tiempos pasados mientras escuchar atención.",
    "Luz tenue velas crear ambiente romántico cena luz luna.",
    "Café caliente reconfortar manos mientras observar lluvia ventana.",
    "Mariposas danzar aire, agregar toque magia prado.",
    "Rugido lejano truenos anunciar tormenta inminente horizonte.",
    "Aroma tierra mojada después lluvia llenar aire frescura.",
    "Hojas crujir bajo pies mientras pasear bosque pleno otoño.",
    "Rincón jardín estar lleno vida visita colibríes mariposas.",
    "Ciudad cobrar vida noche, luces brillantes iluminar rascacielos.",
    "Chirrido suave puertas madera añadir toque nostálgico antiguo caserío.",
    "Neblina matutina envolver paisaje misterio etéreo.",
    "Murmullo arroyo acompañar tranquilo paseo sendero bosque.",
    "Suave tintineo campanillas anunciar llegada brisa primaveral.",
    "Mirada cómplice entre ellos revelar amor resistir paso tiempo.",
    "Ulular búhos romper silencio noche bosque oscuro.",
    "Cometas flotar cielo, pintar estelas colores contra azul claro.",
    "Sabor salado aire marino ser refrescante mientras caminar muelle.",
    "Sombras alargadas árboles bailar suelo atardecer.",
    "Sonido olas acariciar playa ser melodía relajante para oídos.",
    "Risas resonar plaza, gente disfrutar feria pueblo.",
    "Calor sol acariciar piel mientras recostar hierba prado.",
    "Murmullo constante multitud crear ambiente animado mercado.",
    "Luna llena iluminar noche resplandor plateado.",
    "Croar ranas acompañar noche pantano tranquilo.",
    "Aroma lavanda impregnar aire campos violetas bajo sol radiante.",
    "Hojas danzar viento, crear remolino colores otoño."
]


In [38]:
# Oraciones para el entrenamiento (en este caso, usaremos la misma oración como entrada y objetivo)
oraciones_entrenamiento = [oraciones_correctas, oraciones_incorrectas]

In [39]:
# Preprocesar datos
input_sequences, vocab_size, tokenizer = preprocesar_datos(oraciones_entrenamiento)


In [40]:
# Diseñar modelo
max_length = len(input_sequences[0])
modelo_correccion_sintaxis = diseñar_modelo(vocab_size, max_length)

# Entrenar modelo
entrenar_modelo(modelo_correccion_sintaxis, input_sequences, input_sequences, epochs=100)


Epoch 1/100
1/1 - 1s - loss: 4.7868 - accuracy: 0.0000e+00 - 1s/epoch - 1s/step
Epoch 2/100
1/1 - 0s - loss: 4.7840 - accuracy: 0.0339 - 14ms/epoch - 14ms/step
Epoch 3/100
1/1 - 0s - loss: 4.7812 - accuracy: 0.0593 - 14ms/epoch - 14ms/step
Epoch 4/100
1/1 - 0s - loss: 4.7783 - accuracy: 0.1017 - 15ms/epoch - 15ms/step
Epoch 5/100
1/1 - 0s - loss: 4.7753 - accuracy: 0.1186 - 15ms/epoch - 15ms/step
Epoch 6/100
1/1 - 0s - loss: 4.7721 - accuracy: 0.1525 - 14ms/epoch - 14ms/step
Epoch 7/100
1/1 - 0s - loss: 4.7685 - accuracy: 0.1017 - 15ms/epoch - 15ms/step
Epoch 8/100
1/1 - 0s - loss: 4.7646 - accuracy: 0.1102 - 14ms/epoch - 14ms/step
Epoch 9/100
1/1 - 0s - loss: 4.7601 - accuracy: 0.1102 - 15ms/epoch - 15ms/step
Epoch 10/100
1/1 - 0s - loss: 4.7548 - accuracy: 0.0932 - 14ms/epoch - 14ms/step
Epoch 11/100
1/1 - 0s - loss: 4.7484 - accuracy: 0.0932 - 13ms/epoch - 13ms/step
Epoch 12/100
1/1 - 0s - loss: 4.7403 - accuracy: 0.0593 - 14ms/epoch - 14ms/step
Epoch 13/100
1/1 - 0s - loss: 4.7294 

In [41]:
# Corregir gramática
oracion_corregida_gramatica = corregir_gramatica(oracion_original)

# Corregir sintaxis
oracion_corregida_sintaxis = corregir_sintaxis(oracion_corregida_gramatica, modelo_correccion_sintaxis, tokenizer, max_length)


1/1 [==============================] - 0s 221ms/step


In [42]:
# Imprimir resultados
print("Oración Original:", oracion_original)
print("Oración Corregida Gramaticalmente:", oracion_corregida_gramatica)
print("Oración Corregida Sintácticamente:", oracion_corregida_sintaxis)


Oración Original: Ayer, fui al parque a jugar con mis amigos y me olvidé mis sapatos en casa.
Oración Corregida Gramaticalmente: Ayer, fui al parque a jugar con mis amigos y me olvidé mis zapatos en casa.
Oración Corregida Sintácticamente: las olas del océano rompían suavemente en la orilla, creando un sonido relajante. las olas del océano rompían suavemente en la orilla, creando un sonido relajante. las olas del océano rompían suavemente en la orilla, creando un sonido relajante. las olas del océano rompían suavemente en la orilla, creando un sonido relajante. las olas del océano rompían suavemente en la orilla, creando un sonido relajante. las olas del océano rompían suavemente en la orilla, creando un sonido relajante. las olas del océano rompían suavemente en la orilla, creando un sonido relajante. arquitectura antigua pueblo añadir encanto especial calles empedradas. montañas erguir majestuosas, cubiertas manto nieve fresca. montañas erguir majestuosas, cubiertas manto nieve fresc